In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt




In [2]:
raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')

# print(raw.ch_names)

# events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')
# picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')
# tmin, tmax = 1., 4.


# event_id = [ 768, 1791, 2323, 2324, 2325, 2326, 2337, 2339, 3346, 3347, 3348, 3349, 3360, 3362]
# epochs = mne.Epochs(raw, events, None, tmin, tmax, proj=True, picks=picks,
#                 baseline=None, preload=True)




Extracting EDF parameters from k3b.gdf...
GDF file detected
Overlapping events detected. Use find_edf_events for the original events.
Setting channel info structure...
Interpolating stim channel. Events may jitter.
Creating raw.info structure...
Reading 0 ... 986779  =      0.000 ...  3947.116 secs...


<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Overlapping events detected. Use find_edf_events for the original events.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')
<ipython-input-2-3bd60b140d72>:1: RuntimeWarning: Interpolating stim channel. Events may jitter.
  raw = mne.io.read_raw_edf('k3b.gdf', preload=True, stim_channel='auto')


In [3]:
print(raw.ch_names[30])

# 31


In [3]:
events = mne.find_events(raw, shortest_event=0, stim_channel='STI 014')

data = raw.get_data()
print(data.shape)


# raw = mne.io.read_raw_edf("k3b.gdf", preload=True)
_, pos, kind, chan, dur = raw.find_edf_events()
kind_hex = np.array([hex(n) for n in kind])

print(pos[1])
print(pos[2])
print(pos[:20])

print(len(kind_hex))
print(len(pos))
sampling_rate = 250


beep_pos = [pos[i] for i in range(len(kind_hex)) if kind_hex[i] in ['0x301','0x302','0x303','0x304']]
labels = [i[4] for i in kind_hex if i in ['0x301','0x302','0x303','0x304']]
print(labels)


def making_epochs(data):
    epochs = []
    for b in beep_pos:
        epochs.append(data[:,b+sampling_rate*1:b+sampling_rate*4])
    return np.asarray(epochs)


epochs = making_epochs(data)
# dif = [int(pos[i]) - int(pos[i-1]) for i in range(len(beep)-1) ]

# print(dif[:100])

1080 events found
Event IDs: [ 768 1791 2323 2324 2325 2326 2337 2339 3346 3347 3348 3349 3360 3362]
(61, 986780)
2924
2924
[ 2424  2924  2924  3174  4984  5484  5484  5734  7544  7544  8044  8044
  8294 10104 10604 10604 10854 12665 13165 13165]
1502
1502
['3', '4', '3', '3', '3', '2', '4', '2', '1', '4', '1', '2', '2', '1', '2', '4', '1', '2', '3', '1', '1', '3', '1', '3', '1', '1', '1', '2', '4', '3', '2', '4', '2', '2', '3', '4', '1', '3', '2', '4', '3', '1', '4', '1', '4', '3', '1', '1', '2', '3', '1', '4', '4', '2', '2', '3', '4', '2', '1', '4', '4', '3', '1', '3', '4', '3', '3', '3', '1', '4', '2', '4', '4', '2', '1', '1', '4', '1', '3', '2', '4', '2', '1', '4', '2', '2', '4', '3', '1', '2', '2', '4', '2', '3', '4', '4', '3', '3', '4', '1', '4', '2', '3', '2', '1', '3', '4', '1', '2', '1', '3', '1', '3', '4', '3', '3', '1', '4', '3', '4', '4', '2', '2', '1', '2', '4', '4', '3', '3', '1', '4', '1', '3', '1', '2', '2', '3', '3', '3', '1', '1', '2', '2', '2', '1', '1', '2', '2', '1

In [50]:
print(epochs.shape)



(180, 61, 750)


In [53]:
import sklearn 
import numpy as np
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(y_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
(144,)
1.0
Classification accuracy: 0.913889 / Chance level: 0.250000


In [103]:
import pywt

testing_epochs = []
single_epoch = []
samples, chan_num, timesteps = epochs.shape
epochs_power = epochs**2   
# for s in range(samples):
#     single_epoch = []
#     single_epoch.append(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,30,:], 'db4', level=4)[0])
#     single_epoch.append(pywt.wavedec(epochs_power[s,33,:], 'db4', level=4)[0])
#     testing_epochs.append(np.asarray(single_epoch))
    
    
for s in range(samples):
    single_epoch = []
    single_epoch = np.asarray(single_epoch).reshape((1,-1))
    for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
        single_epoch = np.concatenate((single_epoch,np.asarray(xi).reshape((1,-1))), axis = 1)

    testing_epochs.append(single_epoch)
  

testing_epochs = np.asarray(testing_epochs)
print(testing_epochs.shape)
#9540

print(np.count_nonzero(abs(testing_epochs) > 1.0056865244277509e-9))
print(np.min(abs(testing_epochs)))


testing_epochs = testing_epochs.reshape((180, 775))

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(testing_epochs)
# print(scaler.mean_)
testing_epochs = scaler.transform(testing_epochs)


(180, 1, 775)
91
1.0490812995593966e-16


In [89]:
# np.concatenate(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0], pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[1])

y = []
y = np.asarray(y).reshape((1,-1))
for xi in pywt.wavedec(epochs_power[s,27,:], 'db4', level=4):
    y = np.concatenate((y,np.asarray(xi).reshape((1,-1))), axis = 1)
    
# print(pywt.wavedec(epochs_power[s,27,:], 'db4', level=4)[0].shape)
#378 192 99  53  53
print(y.shape)


(1, 775)


In [107]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
from keras import backend as K
import keras
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft
from keras import optimizers


K.clear_session()

# testing_epochs = testing_epochs.reshape((180,1, 775))


class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

history = LossHistory()



testing_epochs = np.asarray(testing_epochs)

# testing_epochs = testing_epochs.swapaxes(1,2)

# time_steps = 53
# n_features = 3

time_steps = 775
n_features = 1



cv = ShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
scores = []
testing_epochs_ANN = testing_epochs.reshape(testing_epochs.shape[0], -1)
cvscores = []

for train_idx, test_idx in cv.split(testing_epochs):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = testing_epochs[train_idx]
    X_test = testing_epochs[test_idx]
    
    

    model = Sequential()
    model.add(Dense(units = 20, input_dim=775, activation="relu", kernel_initializer="uniform"))
    model.add(Dropout(0.2))
    model.add(Dense(units = 5, init='uniform', activation='sigmoid'))
    model.compile( loss = "categorical_crossentropy", 
               optimizer = keras.optimizers.SGD(lr=0.001, clipvalue=0.5), 
               metrics=['accuracy']
             )
    
  #  batch_size cannot be 1, at least contain all the classes for cross-entropy
    model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=36, epochs=1200, callbacks=[history])

    print(model.predict_classes(X_train))
    print(y_train)

    print(model.predict_classes(X_train))
    print(y_train)

    print(history.losses)

    

#     model = Sequential()
#     model.add(LSTM(3, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dropout(0.2))
#     model.add(LSTM(10, return_sequences=False, input_shape=(time_steps, n_features)))
#     model.add(Dense(50, activation='sigmoid'))
#     model.add(Dense(5, activation='sigmoid'))
#     model.compile(loss="categorical_crossentropy",
#                   optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), 
#                   metrics=['accuracy'])
#     model.fit(X_train, keras.utils.np_utils.to_categorical(y_train), batch_size=36, epochs=2000, callbacks=[history])



    scores = model.evaluate(X_test, keras.utils.np_utils.to_categorical(y_test), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    
    
    
print(np.mean(cvscores))







C:\Users\kun\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:56: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=5, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/1200
144/144 [==============================] - 0s 759us/step - loss: 1.6150 - acc: 0.1736
Epoch 2/1200
144/144 [==============================] - 0s 0us/step - loss: 1.6097 - acc: 0.2569
Epoch 3/1200
144/144 [==============================] - 0s 108us/step - loss: 1.6142 - acc: 0.2014
Epoch 4/1200
144/144 [==============================] - 0s 0us/step - loss: 1.6138 - acc: 0.2431
Epoch 5/1200
144/144 [==============================] - 0s 108us/step - loss: 1.6137 - acc: 0.2153
Epoch 6/1200
144/144 [==============================] - 0s 0us/step - loss: 1.6105 - acc: 0.2292
Epoch 7/1200
144/144 [==============================] - 0s 0us/step - loss: 1.6105 - acc: 0.2639
Epoch 8/1200
144/144 [==============================] - 0s 108us/step - loss: 1.6107 - acc: 0.2431
Epoch 9/1200
144/144 [==============================] - 0s 0us/step - loss: 1.6128 - acc: 0.2361
Epoch 10/1200
144/144 [==============================] - 0s 108us/step - loss: 1.6115 - acc: 0.2292
Epoch 11/1200
144/1

144/144 [==============================] - 0s 0us/step - loss: 1.5952 - acc: 0.3125
Epoch 85/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5923 - acc: 0.3125
Epoch 86/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5956 - acc: 0.2708
Epoch 87/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5917 - acc: 0.3264
Epoch 88/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5953 - acc: 0.3333
Epoch 89/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5954 - acc: 0.2708
Epoch 90/1200
144/144 [==============================] - 0s 109us/step - loss: 1.5929 - acc: 0.3056
Epoch 91/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5942 - acc: 0.2708
Epoch 92/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5932 - acc: 0.3125
Epoch 93/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5930 - acc: 0.3194
Epoch 94/1200
144/144 [===

144/144 [==============================] - 0s 108us/step - loss: 1.5771 - acc: 0.3958
Epoch 167/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5769 - acc: 0.3333
Epoch 168/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5755 - acc: 0.4097
Epoch 169/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5764 - acc: 0.3750
Epoch 170/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5780 - acc: 0.3819
Epoch 171/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5754 - acc: 0.3958
Epoch 172/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5791 - acc: 0.3611
Epoch 173/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5768 - acc: 0.4028
Epoch 174/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5777 - acc: 0.3542
Epoch 175/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5764 - acc: 0.3611
Epoch 176/1200
14

144/144 [==============================] - 0s 0us/step - loss: 1.5619 - acc: 0.4444
Epoch 249/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5587 - acc: 0.4514
Epoch 250/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5626 - acc: 0.3889
Epoch 251/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5602 - acc: 0.3681
Epoch 252/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5579 - acc: 0.4583
Epoch 253/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5602 - acc: 0.4375
Epoch 254/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5548 - acc: 0.4861
Epoch 255/1200
144/144 [==============================] - 0s 108us/step - loss: 1.5588 - acc: 0.4514
Epoch 256/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5609 - acc: 0.4514
Epoch 257/1200
144/144 [==============================] - 0s 0us/step - loss: 1.5605 - acc: 0.4236
Epoch 258/1200
144/

...

144/144 [==============================] - 0s 83us/step - loss: 1.3927 - acc: 0.6875
Epoch 900/1200
144/144 [==============================] - 0s 56us/step - loss: 1.3973 - acc: 0.7639
Epoch 901/1200
144/144 [==============================] - 0s 56us/step - loss: 1.3927 - acc: 0.6944
Epoch 902/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3940 - acc: 0.6875
Epoch 903/1200
144/144 [==============================] - 0s 115us/step - loss: 1.3924 - acc: 0.7292
Epoch 904/1200
144/144 [==============================] - 0s 38us/step - loss: 1.3920 - acc: 0.7222
Epoch 905/1200
144/144 [==============================] - 0s 87us/step - loss: 1.3901 - acc: 0.7292
Epoch 906/1200
144/144 [==============================] - 0s 32us/step - loss: 1.3877 - acc: 0.7153
Epoch 907/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3910 - acc: 0.7361
Epoch 908/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3889 - acc: 0.7083
Epoch 909/1200
14

144/144 [==============================] - 0s 0us/step - loss: 1.3559 - acc: 0.7847
Epoch 982/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3695 - acc: 0.7014
Epoch 983/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3700 - acc: 0.7222
Epoch 984/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3578 - acc: 0.7986
Epoch 985/1200
144/144 [==============================] - 0s 109us/step - loss: 1.3591 - acc: 0.7500
Epoch 986/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3609 - acc: 0.7361
Epoch 987/1200
144/144 [==============================] - 0s 108us/step - loss: 1.3606 - acc: 0.7292
Epoch 988/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3588 - acc: 0.7361
Epoch 989/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3594 - acc: 0.7083
Epoch 990/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3591 - acc: 0.7569
Epoch 991/1200
144/14

144/144 [==============================] - 0s 109us/step - loss: 1.3359 - acc: 0.7569
Epoch 1063/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3263 - acc: 0.7778
Epoch 1064/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3377 - acc: 0.7639
Epoch 1065/1200
144/144 [==============================] - 0s 108us/step - loss: 1.3269 - acc: 0.7778
Epoch 1066/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3325 - acc: 0.7778
Epoch 1067/1200
144/144 [==============================] - 0s 108us/step - loss: 1.3300 - acc: 0.7361
Epoch 1068/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3271 - acc: 0.8056
Epoch 1069/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3210 - acc: 0.8194
Epoch 1070/1200
144/144 [==============================] - 0s 108us/step - loss: 1.3274 - acc: 0.7569
Epoch 1071/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3291 - acc: 0.7431
Epoch 10

144/144 [==============================] - 0s 0us/step - loss: 1.3004 - acc: 0.7986
Epoch 1144/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3051 - acc: 0.7361
Epoch 1145/1200
144/144 [==============================] - 0s 0us/step - loss: 1.2889 - acc: 0.7708
Epoch 1146/1200
144/144 [==============================] - 0s 0us/step - loss: 1.2950 - acc: 0.7500
Epoch 1147/1200
144/144 [==============================] - 0s 108us/step - loss: 1.2957 - acc: 0.8056
Epoch 1148/1200
144/144 [==============================] - 0s 0us/step - loss: 1.2949 - acc: 0.7569
Epoch 1149/1200
144/144 [==============================] - 0s 0us/step - loss: 1.3031 - acc: 0.7708
Epoch 1150/1200
144/144 [==============================] - 0s 0us/step - loss: 1.2900 - acc: 0.7986
Epoch 1151/1200
144/144 [==============================] - 0s 0us/step - loss: 1.2898 - acc: 0.7639
Epoch 1152/1200
144/144 [==============================] - 0s 108us/step - loss: 1.2961 - acc: 0.7431
Epoch 1153/1

acc: 50.00%
50.0


In [23]:
weights, biases = model.layers[0].get_weights()
print(np.asarray(weights)[:,0])

[-0.0207012   0.03105569  0.02335299 -0.00731922  0.03082303 -0.0453177
  0.04096075  0.0152482  -0.04893824 -0.03698607 -0.04089217  0.03575618
  0.04030136 -0.02033446  0.02710366  0.0423562  -0.03371464  0.02415731
  0.03435538 -0.03418693 -0.03781922 -0.0142321  -0.00151715  0.02819661
  0.01891855  0.04658123  0.02832549 -0.01705701 -0.00835256  0.00221534
 -0.04398263  0.0117085   0.0248772  -0.06454493  0.01345973  0.03564309
 -0.03374089  0.04120235  0.02626074 -0.01899987  0.02920159  0.02039359
  0.04051714  0.01622102  0.00131551 -0.00376901  0.00654611 -0.03208773
  0.02378325  0.01935382 -0.03526055 -0.00592585  0.01088757 -0.04603091
  0.03392961  0.0481873   0.04271042  0.04083136 -0.01266236 -0.03547047
 -0.00520672 -0.0183892   0.03050713 -0.03129324  0.04606132  0.04971017
  0.02748834 -0.00698207  0.01648488  0.02683713  0.00957265  0.01224351
  0.02448628 -0.02667756 -0.04241735 -0.03869012  0.03481081 -0.03404626
  0.02150485 -0.02516186  0.01786185 -0.01443924  0.

In [85]:
print(model.predict_classes(X_test))
print(y_test)



[2 1 1 4 3 2 2 2 1 3 4 1 4 2 4 2 4 2 2 2 1 4 2 1 4 1 4 2 2 2 1 3 4 4 3 3]
['1' '4' '3' '3' '1' '4' '1' '1' '4' '3' '3' '2' '4' '2' '3' '1' '4' '4'
 '3' '4' '3' '4' '2' '2' '4' '4' '4' '3' '1' '2' '1' '3' '1' '4' '3' '3']


In [ ]:
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
print(scaler.fit(data))
StandardScaler(copy=True, with_mean=True, with_std=True)
print(scaler.mean_)
print(scaler.transform(data))
